One-hot encoded 6-bit utterance vector for 3 colors and 2 shapes
New AE trained on 3 colors and 2 shapes

In [ ]:
from MuJoCo_Gym.mujoco_rl import MuJoCoRL
from MuJoCo_Gym.wrappers import GymnasiumWrapper, GymWrapper
from gymnasium.wrappers.frame_stack import FrameStack
from gymnasium.experimental.wrappers import NormalizeObservationV0
from dynamics import *
import argparse
import os
import random
import time
from distutils.util import strtobool
import gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions.normal import Normal
from torch.utils.tensorboard import SummaryWriter

from wrappers.record_episode_statistics import RecordEpisodeStatistics
from wrappers.frame_stack import FrameStack
from wrappers.normalizeObservation import NormalizeObservation
from wrappers.normalizeRewards import NormalizeReward

from progressbar import progressbar
import tensorflow as tf

# Check if eager execution is enabled
if not tf.executing_eagerly():
    tf.compat.v1.enable_eager_execution()

# Dynamics

## Vision

In [2]:
from autoencoder import load_autoencoder_model, get_a_single_image_embedding

class Image:
    def __init__(self, environment):
        self.environment = environment
        self.observation_space = {"low": [0 for _ in range(50)], "high": [255 for _ in range(50)]}
        self.action_space = {"low": [], "high": []}
        self.autoencoder = load_autoencoder_model("models/3colors_2shapes_50.tf")
        self.index = 0

    def dynamic(self, agent, actions):
        self.index = self.index + 1
        image = self.environment.get_camera_data(agent + "_camera")
        result = get_a_single_image_embedding(self.autoencoder, image)
        # cv2.imwrite(f"/Users/cowolff/Documents/GitHub/s.language_experiments/images/{agent}_{self.index}.png", image)
        return 0, result, False, {}

## Communication

In [3]:
def get_shape(shape_id):
    if shape_id == 6:  # box
        return np.array([1, 0])
    elif shape_id == 2:  # sphere
        return np.array([0, 1])
    else:
        raise ValueError("Unknown shape")
    
def encode_to_one_hot_vector(color, shape):
    combined_vector = [0, 0, 0, 0, 0, 0]
    color_index = np.argmax(color)
    shape_index = np.argmax(shape)
    # [Red Box, Red Sphere, Green Box, Green Sphere, Blue Box, Blue Sphere]
    combined_index = color_index * 2 + shape_index
    combined_vector[combined_index] = 1
    return combined_vector

In [4]:
class Communication:
    def __init__(self, environment):
        self.environment = environment
        self.observation_space = {"low": [0, 0, 0, 0, 0, 0], "high": [1, 1, 1, 1, 1, 1]}
        self.action_space = {"low": [0, 0, 0, 0, 0, 0], "high": [1, 1, 1, 1, 1, 1]}

    def dynamic(self, agent, actions):
        if "utterance" not in self.environment.data_store[agent].keys():
            self.environment.data_store[agent]["utterance"] = None
        if agent == "receiver":
            utterance = [0, 0, 0, 0, 0, 0]
            if "target_color" in self.environment.data_store.keys() and "target_shape" in self.environment.data_store.keys():
                utterance = encode_to_one_hot_vector(self.environment.data_store["target_color"], self.environment.data_store["target_shape"])
            observation = utterance
            # print(f'Comm Receiver Utterance: {utterance}')  # TODO remove
        elif agent == "sender":
            utterance = [0, 0, 0, 0, 0, 0]
            utterance[np.argmax(actions)] = 1
            self.environment.data_store[agent]["utterance"] = actions
            self.environment.data_store[agent]["utterance_max"] = utterance
            # print(f'Comm Sender Utterance: {actions}, Utterance_max: {utterance}')  # TODO remove
            observation = [0, 0, 0, 0, 0, 0]
        else:
            print("Unknown agent")
        return 0, observation, False, {}

## Reward Dynamic Function
The reward function for the receiver gives a positive reward for the agent to move closer to the current target object.<br/>
The sender in turn gets a positive reward if it chooses the correct word encoding for the color and shape of the current target object.

In [5]:
from sklearn.metrics import mean_squared_error
from autoencoder import Autoencoder
import math
import mujoco

In [6]:
class Reward:
    def __init__(self, environment):
        self.environment = environment
        self.observation_space = {"low": [], "high": []}
        self.action_space = {"low": [], "high": []}
        self.choices = ["choice_1", "choice_2"]

    def dynamic(self, agent, actions):
        if not "target" in self.environment.data_store.keys():
            color = self.environment.get_data("reference_geom")["color"]
            shape = get_shape(self.environment.get_data("reference_geom")["shape"])
            # print(f'Color: {color}, Shape: {shape}')  # TODO remove
            for choice in self.choices:
                choice_color = self.environment.get_data(choice + "_geom")["color"]
                choice_shape = get_shape(self.environment.get_data(choice + "_geom")["shape"])
                # Check if both color and shape match
                if (color == choice_color).all() and (shape == choice_shape).all():
                    self.environment.data_store["target"] = choice
                    self.environment.data_store["target_color"] = choice_color
                    self.environment.data_store["target_shape"] = choice_shape
                    self.environment.data_store["last_distance"] = copy.deepcopy(self.environment.distance("receiver_geom", choice + "_geom"))
                    break 
        if agent == "receiver":
            target = self.environment.data_store["target"]
            new_distance = self.environment.distance("receiver_geom", target + "_geom")
            reward = (self.environment.data_store["last_distance"] - new_distance) * 10
            self.environment.data_store["last_distance"] = copy.deepcopy(new_distance)
        elif agent == "sender":
            reference = [0, 0, 0, 0, 0, 0]
            color = self.environment.data_store["target_color"]
            shape = self.environment.data_store["target_shape"]
            
            reference = encode_to_one_hot_vector(color, shape)
            reward = 0
            if "utterance" in self.environment.data_store[agent].keys():
                reward = -1 * mean_squared_error(reference, self.environment.data_store[agent]["utterance"])
        return reward, [], False, {}

## Accuracy Measurement Dynamic

In [7]:
class Accuracy:
    def __init__(self, environment):
        self.environment = environment
        self.observation_space = {"low": [], "high": []}
        self.action_space = {"low": [], "high": []}
        self.accuracies = []
        self.variances = []
        self.sendAccuracies = []
        self.sendVariances = []
        self.currentSend = []
        self.report_accuracy = {"sender": 0, "receiver": 0}

    def dynamic(self, agent, actions):
        choices = ["choice_1", "choice_2"]
        variance = {"choice_1":1, "choice_2":-1}
        if "target" in self.environment.data_store.keys():
            if "sendVariances" not in self.environment.data_store.keys():
                self.environment.data_store["sendVariances"] = True
                self.currentSend = [0, 0, 0, 0, 0, 0]
            target = self.environment.data_store["target"]
            # if any(self.environment.collision(ankle, target + "_geom") for ankle in ["left_leg_geom_2", "left_ankle_geom_2", "right_leg_geom_2", "right_ankle_geom_2", "back_leg_geom_2", "third_ankle_geom_2", "rightback_leg_geom_2", "fourth_ankle_geom_2"]):
            if self.environment.collision("receiver_geom", target + "_geom"):
                self.accuracies.append(1)
                self.variances.append(variance[target])

                if len(self.variances) > 50:
                    report_variance = 1 - abs(sum(self.variances[-50:]) / 50)
                    self.report_accuracy["receiver"] = sum(self.accuracies[-50:]) / 50
            # elif any(self.environment.collision(ankle, [choice for choice in choices if choice != target][0] + "_geom") for ankle in ["left_leg_geom_2", "left_ankle_geom_2", "right_leg_geom_2", "right_ankle_geom_2", "back_leg_geom_2", "third_ankle_geom_2", "rightback_leg_geom_2", "fourth_ankle_geom_2"]):
            elif self.environment.collision("receiver_geom", [choice for choice in choices if choice != target][0] + "_geom"):
                self.accuracies.append(0)
                self.variances.append(variance[[choice for choice in choices if choice != target][0]])

                if len(self.variances) > 50:
                    report_variance = 1 - abs(sum(self.variances[-50:]) / 50)
                    self.report_accuracy[0] = sum(self.accuracies[-50:]) / 50
            if "utterance_max" in self.environment.data_store[agent].keys():
                color = self.environment.data_store["target_color"]
                shape = self.environment.data_store["target_shape"]
                reference = encode_to_one_hot_vector(color, shape)
                self.currentSend = np.add(self.currentSend, self.environment.data_store[agent]["utterance_max"])
                # print(f'Utterance Max: {self.environment.data_store[agent]["utterance_max"]}')  # TODO remove
                # print(f'Reference: {reference}')  # TODO remove

                if self.environment.data_store[agent]["utterance_max"]  == reference:
                    self.sendAccuracies.append(1)
                else:
                    self.sendAccuracies.append(0)
        return 0, [], False, {}

# Reward Functions

In [8]:
def target_reward(mujoco_gym, agent):
    if agent == "receiver":
        choices = ["choice_1", "choice_2"]
        if not "target" in mujoco_gym.data_store.keys():
                color = mujoco_gym.get_data("reference_geom")["color"]
                shape = get_shape(mujoco_gym.get_data("reference_geom")["shape"])
                # print(f'Target Reward Color: {color}, Shape: {shape}')  # TODO remove
                for choice in choices:
                    choice_shape = get_shape(mujoco_gym.get_data(choice + "_geom")["shape"])
                    if (color == mujoco_gym.get_data(choice + "_geom")["color"]).all() and (shape == choice_shape).all():
                        mujoco_gym.data_store["target"] = choice
                        mujoco_gym.data_store["target_color"] = mujoco_gym.get_data(choice + "_geom")["color"]
                        mujoco_gym.data_store["target_shape"] = choice_shape
                        # print(f'Target Reward target shape: {choice_shape}')  # TODO remove
                        break
        
        target = mujoco_gym.data_store["target"]
        # for ankle in ["left_leg_geom_2", "left_ankle_geom_2", "right_leg_geom_2", "right_ankle_geom_2", "back_leg_geom_2", "third_ankle_geom_2", "rightback_leg_geom_2", "fourth_ankle_geom_2"]:
        for ankle in ["receiver_geom"]:
            if mujoco_gym.collision(ankle, target + "_geom"):
                return 1
            elif mujoco_gym.collision(ankle, [choice for choice in choices if choice != target][0] + "_geom"):
                return -1
    return 0

In [9]:
def collision_reward(mujoco_gym, agent):
    for border in ["border1_geom", "border2_geom", "border3_geom", "border4_geom", "border5_geom"]:
        # for ankle in ["left_leg_geom_2", "left_ankle_geom_2", "right_leg_geom_2", "right_ankle_geom_2", "back_leg_geom_2", "third_ankle_geom_2", "rightback_leg_geom_2", "fourth_ankle_geom_2"]:
        for ankle in [agent + "_geom"]:
            if mujoco_gym.collision(border, ankle):
                return -0.1
    return 0

# Environment Setup

In [10]:
def make_env(config_dict):
    def thunk():
        window = 5
        env = MuJoCoRL(config_dict=config_dict)
        # env = GymWrapper(env, "receiver")
        # env = FrameStack(env, 4)
        env = NormalizeObservation(env)
        env = NormalizeReward(env)
        # env = RecordEpisodeStatistics(env)
        return env

    return thunk

In [11]:
# xml_files = ["levels/" + file for file in os.listdir("levels/")]
xml_files = ["levels_shape/modified/" + file for file in os.listdir("levels_shape/modified/")]
agents = ["sender", "receiver"]

config_dict = {"xmlPath":xml_files, 
                   "agents":agents, 
                   "rewardFunctions":[collision_reward, target_reward], 
                   "doneFunctions":[target_done, border_done], 
                   "skipFrames":5,
                   "environmentDynamics":[Image, Reward, Communication, Accuracy],
                   "freeJoint":True,
                   "renderMode":False,
                   "maxSteps":1024,
                   "agentCameras":True}

env = make_env(config_dict)()

/Users/imtezcan/Repositories/CogSci/EBIMAS/venv-ebimas/lib/python3.10/site-packages/gymnasium/spaces/box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float32
  gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


# RL Model

In [12]:
def layer_init(layer, std=np.sqrt(2), bias_const=0.0):
    torch.nn.init.orthogonal_(layer.weight, std)
    torch.nn.init.constant_(layer.bias, bias_const)
    return layer

class Agent(nn.Module):
    def __init__(self, envs):
        super(Agent, self).__init__()
        self.critic = nn.Sequential(
            nn.Flatten(),
            layer_init(nn.Linear(np.array(envs.observation_space.shape).prod(), 128)),
            nn.Tanh(),
            layer_init(nn.Linear(128, 128)),
            nn.Tanh(),
            layer_init(nn.Linear(128, 1), std=1.0),
        )
        self.actor_mean = nn.Sequential(
            nn.Flatten(),
            layer_init(nn.Linear(np.array(envs.observation_space.shape).prod(), 128)),
            nn.Tanh(),
            layer_init(nn.Linear(128, 128)),
            nn.Tanh(),
            layer_init(nn.Linear(128, np.prod(envs.action_space.shape)), std=0.01),
        )
        self.actor_logstd = nn.Parameter(torch.zeros(1, np.prod(envs.action_space.shape)))

    def get_value(self, x):
        return self.critic(x)

    def get_action_and_value(self, x, action=None):
        action_mean = self.actor_mean(x)
        action_logstd = self.actor_logstd.expand_as(action_mean)
        action_std = torch.exp(action_logstd)
        probs = Normal(action_mean, action_std)
        if action is None:
            action = probs.sample()
        return action, probs.log_prob(action).sum(1), probs.entropy().sum(1), self.critic(x)

# Buffer

In [13]:
class Buffer():
    def __init__(self, num_steps, envs, num_envs, device):
        self.obs = torch.zeros((num_steps, num_envs) + envs.observation_space.shape).to(device)
        self.actions = torch.zeros((num_steps, num_envs) + envs.action_space.shape).to(device)
        self.logprobs = torch.zeros((num_steps, num_envs)).to(device)
        self.rewards = torch.zeros((num_steps, num_envs)).to(device)
        self.dones = torch.zeros((num_steps, num_envs)).to(device)
        self.values = torch.zeros((num_steps, num_envs)).to(device)

# Update Function

In [14]:
def update_agent(agent, buffer, optimizer, next_obs, next_done, env, batch_size, update_epochs, minibatch_size, clip_coef, vf_coef, ent_coef, max_grad_norm, target_kl, clip_vloss, norm_adv, gae_lambda, gae, gamma, device, num_steps):

    with torch.no_grad():
        next_value = agent.get_value(next_obs).reshape(1, -1)
        if gae:
            advantages = torch.zeros_like(buffer.rewards).to(device)
            lastgaelam = 0
            for t in reversed(range(num_steps)):
                if t == num_steps - 1:
                    nextnonterminal = 1.0 - next_done
                    nextvalues = next_value
                else:
                    nextnonterminal = 1.0 - buffer.dones[t + 1]
                    nextvalues = buffer.values[t + 1]
                delta = buffer.rewards[t] + gamma * nextvalues * nextnonterminal - buffer.values[t]
                advantages[t] = lastgaelam = delta + gamma * gae_lambda * nextnonterminal * lastgaelam
            returns = advantages + buffer.values
        else:
            returns = torch.zeros_like(buffer.rewards).to(device)
            for t in reversed(range(num_steps)):
                if t == num_steps - 1:
                    nextnonterminal = 1.0 - next_done
                    next_return = next_value
                else:
                    nextnonterminal = 1.0 - buffer.dones[t + 1]
                    next_return = returns[t + 1]
                returns[t] = buffer.rewards[t] + gamma * nextnonterminal * next_return
            advantages = returns - buffer.values

    # flatten the batch
    b_obs = buffer.obs.reshape((-1,) + env.observation_space.shape)
    b_logprobs = buffer.logprobs.reshape(-1)
    b_actions = buffer.actions.reshape((-1,) + env.action_space.shape)
    b_advantages = advantages.reshape(-1)
    b_returns = returns.reshape(-1)
    b_values = buffer.values.reshape(-1)

    # Optimizing the policy and value network
    b_inds = np.arange(batch_size)
    clipfracs = []
    for epoch in range(update_epochs):
        np.random.shuffle(b_inds)
        for start in range(0, batch_size, minibatch_size):
            end = start + minibatch_size
            mb_inds = b_inds[start:end]

            _, newlogprob, entropy, newvalue = agent.get_action_and_value(b_obs[mb_inds], b_actions[mb_inds])
            logratio = newlogprob - b_logprobs[mb_inds]
            ratio = logratio.exp()

            with torch.no_grad():
                # calculate approx_kl http://joschu.net/blog/kl-approx.html
                old_approx_kl = (-logratio).mean()
                approx_kl = ((ratio - 1) - logratio).mean()
                clipfracs += [((ratio - 1.0).abs() > clip_coef).float().mean().item()]

            mb_advantages = b_advantages[mb_inds]
            if norm_adv:
                mb_advantages = (mb_advantages - mb_advantages.mean()) / (mb_advantages.std() + 1e-8)

            # Policy loss
            pg_loss1 = -mb_advantages * ratio
            pg_loss2 = -mb_advantages * torch.clamp(ratio, 1 - clip_coef, 1 + clip_coef)
            pg_loss = torch.max(pg_loss1, pg_loss2).mean()

            # Value loss
            newvalue = newvalue.view(-1)
            if clip_vloss:
                v_loss_unclipped = (newvalue - b_returns[mb_inds]) ** 2
                v_clipped = b_values[mb_inds] + torch.clamp(
                    newvalue - b_values[mb_inds],
                    -clip_coef,
                    clip_coef,
                )
                v_loss_clipped = (v_clipped - b_returns[mb_inds]) ** 2
                v_loss_max = torch.max(v_loss_unclipped, v_loss_clipped)
                v_loss = 0.5 * v_loss_max.mean()
            else:
                v_loss = 0.5 * ((newvalue - b_returns[mb_inds]) ** 2).mean()

            entropy_loss = entropy.mean()
            loss = pg_loss - ent_coef * entropy_loss + v_loss * vf_coef

            optimizer.zero_grad()
            loss.backward()
            nn.utils.clip_grad_norm_(agent.parameters(), max_grad_norm)
            optimizer.step()

        if target_kl is not None:
            if approx_kl > target_kl:
                break

    y_pred, y_true = b_values.cpu().numpy(), b_returns.cpu().numpy()
    var_y = np.var(y_true)
    explained_var = np.nan if var_y == 0 else 1 - np.var(y_true - y_pred) / var_y

# Training

## Helper Function

In [15]:
def initialize_agent(env, device, learning_rate):
    agent = Agent(env).to(device)
    optimizer = optim.Adam(agent.parameters(), lr=learning_rate, eps=1e-5)
    return agent, optimizer

def get_action_and_update_buffer(agent, obs, buffer, step):
    with torch.no_grad():
        action, logprob, _, value = agent.get_action_and_value(obs)
        buffer.values[step] = value.flatten()
    buffer.actions[step] = action
    buffer.logprobs[step] = logprob
    return action

def reset_environment(env, device):
    next_obs, infos = env.reset()
    next_obs = {k: torch.Tensor(v).unsqueeze(0).to(device) for k, v in next_obs.items()}
    return next_obs, infos

## Hyperparameters

In [17]:
exp_name = "3 Colors 2 Shapes New AE 6-bit Vector"

learning_rate = 1e-5
seed = 1
# total_timesteps = 20000000
total_timesteps = 1000000
torch_deterministic = True
cuda = False
mps = True
track = False
wandb_project_name = "ppo-implementation-details"
wandb_entity = None
capture_video = False

# Algorithm-specific arguments
num_envs = 1
num_steps = 2048
anneal_lr = True
gae = True
gamma = 0.99
gae_lambda = 0.95
num_minibatches = 128
update_epochs = 10
norm_adv = True
clip_coef = 0.2
clip_vloss = True
ent_coef = 0.0
vf_coef = 0.5
max_grad_norm = 0.5
target_kl = None
store_freq = 20

# Calculate derived variables
batch_size = int(num_envs * num_steps)
minibatch_size = int(batch_size // num_minibatches)

In [18]:
torch.set_default_dtype(torch.float32)

run_name = f"{exp_name}__{seed}__{int(time.time())}"

writer = SummaryWriter(f"runs/{run_name}")

writer.add_text("environment/level_number", str(len(xml_files)), 0)
writer.add_text("environment/agents", ', '.join(agents), 0)
writer.add_text("hyperparameters/learning_rate", str(learning_rate), 0)
writer.add_text("hyperparameters/network_size", ', '.join(str(e) for e in [512, 256]), 0)
writer.add_text("hyperparameters/batch", str(minibatch_size), 0)

# TRY NOT TO MODIFY: seeding
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = torch_deterministic

device = torch.device("cuda" if torch.cuda.is_available() and cuda else "cpu")

obs, infos = env.reset()

sender, sender_optimizer = initialize_agent(env, device, learning_rate)
receiver, receiver_optimizer = initialize_agent(env, device, learning_rate)

buffer_sender = Buffer(num_steps, env, num_envs, device)
buffer_receiver = Buffer(num_steps, env, num_envs, device)

global_step = 0
start_time = time.time()
next_obs, infos = reset_environment(env, device)

next_done = {"sender": torch.zeros(num_envs).to(device), "receiver": torch.zeros(num_envs).to(device)}

num_updates = total_timesteps // batch_size
train_start = time.time()

epoch_lengths = []
current_length = 0

for update in progressbar(range(1, num_updates + 1), redirect_stdout=True):
    # Annealing the rate if instructed to do so.
    if anneal_lr:
        frac = 1.0 - (update - 1.0) / num_updates
        lrnow = frac * learning_rate
        sender_optimizer.param_groups[0]["lr"] = lrnow
        receiver_optimizer.param_groups[0]["lr"] = lrnow
    
    epoch_rewards = {"sender":0, "receiver":0}
    current_rewards = {"sender":[], "receiver":[]}
    variances = {"sender":[], "receiver":[]}
    epoch_runs = 0
    episode_accuracies = 0
    episode_sendAccuracies = 0
    for step in range(0, num_steps):
        global_step += 1 * num_envs
        current_length += 1
        buffer_sender.obs[step] = next_obs["sender"]
        buffer_receiver.obs[step] = next_obs["receiver"]


        sender_action = get_action_and_update_buffer(sender, next_obs["sender"], buffer_sender, step)
        receiver_action = get_action_and_update_buffer(receiver, next_obs["receiver"], buffer_receiver, step)

        next_obs, reward, terminations, truncations, info = env.step({"sender": sender_action.cpu().numpy()[0], "receiver": receiver_action.cpu().numpy()[0]})
        current_rewards["sender"].append(reward["sender"])
        current_rewards["receiver"].append(reward["receiver"])
        next_obs = {"sender": torch.Tensor(next_obs["sender"]).unsqueeze(0).to(device), "receiver": torch.Tensor(next_obs["receiver"]).unsqueeze(0).to(device)}

        if terminations["sender"] or terminations["receiver"] or truncations["sender"] or truncations["receiver"]:
            next_obs, infos = reset_environment(env, device)
            epoch_rewards["sender"] += sum(current_rewards["sender"])
            epoch_rewards["receiver"] += sum(current_rewards["receiver"])

            epoch_lengths.append(current_length)
            current_length = 0

            dynamic = env.env.env.environment_dynamics[3]

            if len(dynamic.sendAccuracies) > 512:
                episode_sendAccuracies = sum(dynamic.sendAccuracies[-512:]) / 512
                del dynamic.sendAccuracies[:-513]
                writer.add_scalar("charts/sender/accuracies", episode_sendAccuracies, global_step)

            if len(dynamic.accuracies) > 4:
                window = min(15, len(dynamic.accuracies))
                episode_accuracies = sum(dynamic.accuracies[-1 * window:]) / window
                writer.add_scalar("charts/receiver/accuracies", episode_accuracies, global_step)
                if window == 15:
                    del dynamic.accuracies[:-16]

            if len(dynamic.variances) > 4:
                window = min(15, len(dynamic.variances))
                current_variance = sum(dynamic.variances[-1 * window:]) / window
                writer.add_scalar("charts/receiver_variance", current_variance, global_step)
                if window == 15:
                    del dynamic.variances[:-16]

            if len(epoch_lengths) > 3:
                window = min(10, len(epoch_lengths))
                epoch_length = sum(epoch_lengths[-1 * window:]) / window
                writer.add_scalar("charts/episodic_length", epoch_length, global_step)
                if window == 10:
                    del epoch_lengths[:-11]
            epoch_runs += 1
        
        buffer_sender.rewards[step] = torch.tensor(reward["sender"]).to(device).view(-1)
        buffer_receiver.rewards[step] = torch.tensor(reward["receiver"]).to(device).view(-1)
        next_done = {"sender": torch.Tensor([terminations["sender"]]).to(device), "receiver": torch.Tensor([terminations["receiver"]]).to(device)}
    if update % store_freq == 0:
        torch.save(sender, "models/model" + str(start_time) + ".pth")
        torch.save(receiver, "models/model" + str(start_time) + ".pth")

    update_agent(sender, buffer_sender, sender_optimizer, next_obs["sender"], next_done["sender"], env, batch_size, update_epochs, minibatch_size, clip_coef, vf_coef, ent_coef, max_grad_norm, target_kl, clip_vloss, norm_adv, gae_lambda, gae, gamma, device, step)
    update_agent(receiver, buffer_receiver, receiver_optimizer, next_obs["receiver"], next_done["receiver"], env, batch_size, update_epochs, minibatch_size, clip_coef, vf_coef, ent_coef, max_grad_norm, target_kl, clip_vloss, norm_adv, gae_lambda, gae, gamma, device, step)

    writer.add_scalar("charts/learning_rate", sender_optimizer.param_groups[0]["lr"], global_step)
    writer.add_scalar("charts/sender/episodic_return", epoch_rewards["sender"] / epoch_runs, global_step)
    writer.add_scalar("charts/receiver/episodic_return", epoch_rewards["receiver"] / epoch_runs, global_step)
    print("SPS:", int(global_step / (time.time() - start_time)), "Average Reward:", epoch_rewards["sender"] / epoch_runs)
    writer.add_scalar("charts/SPS", int(global_step / (time.time() - start_time)), global_step)

torch.save(sender, "models/model" + str(start_time) + ".pth")
torch.save(receiver, "models/model" + str(start_time) + ".pth")
writer.close()

  0% (0 of 1464) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--

SPS: 52 Average Reward: -40.65240528976611


  0% (1 of 1464) |                       | Elapsed Time: 0:00:38 ETA:  15:49:01

SPS: 57 Average Reward: -11.090793909216826


  0% (2 of 1464) |                       | Elapsed Time: 0:01:11 ETA:  13:10:55

...

 99% (1462 of 1464) |################## | Elapsed Time: 12:36:32 ETA:   0:01:01

SPS: 65 Average Reward: -4.964068058396707


 99% (1463 of 1464) |################## | Elapsed Time: 12:37:03 ETA:   0:00:31

SPS: 65 Average Reward: -4.473762393103606


100% (1464 of 1464) |###################| Elapsed Time: 12:37:34 Time: 12:37:34


In [19]:
torch.save(sender, "models/sender" + str(start_time) + ".pth")
torch.save(receiver, "models/receiver" + str(start_time) + ".pth")

# Direct communication test

In [19]:
class Test_Communication:
    def __init__(self, environment):
        self.environment = environment
        self.observation_space = {"low": [0, 0, 0, 0, 0, 0], "high": [1, 1, 1, 1, 1, 1]}
        self.action_space = {"low": [0, 0, 0, 0, 0, 0], "high": [1, 1, 1, 1, 1, 1]}

    def dynamic(self, agent, actions):
        if "utterance" not in self.environment.data_store[agent].keys():
            self.environment.data_store[agent]["utterance"] = None
        if agent == "receiver":
            utterance = [0, 0, 0, 0, 0, 0]
            if "utterance_max" in self.environment.data_store["sender"].keys():
                observation = self.environment.data_store["sender"]["utterance_max"]
            else:
                observation = utterance
        elif agent == "sender":
            utterance = [0, 0, 0, 0, 0, 0]
            utterance[np.argmax(actions)] = 1
            self.environment.data_store[agent]["utterance"] = actions
            self.environment.data_store[agent]["utterance_max"] = utterance
            observation = [0, 0, 0, 0, 0, 0]
        else:
            print("Dafaq is going on here?")
        return 0, observation, False, {}

In [18]:
sender = torch.load("models/sender1709758854.6746309.pth")
receiver = torch.load("models/receiver1709758854.6746309.pth")

In [20]:
xml_files = ["levels_shape_backup/modified/" + file for file in os.listdir("levels_shape_backup/modified/")]
agents = ["sender", "receiver"]

config_dict = {"xmlPath":xml_files, 
                   "agents":agents, 
                   "rewardFunctions":[collision_reward, target_reward], 
                   "doneFunctions":[target_done, border_done], 
                   "skipFrames":5,
                   "environmentDynamics":[Image, Reward, Test_Communication, Accuracy],
                   "freeJoint":True,
                   "renderMode":False,
                   "maxSteps":1024,
                   "agentCameras":True}

env = make_env(config_dict)()

/Users/imtezcan/Repositories/CogSci/EBIMAS/venv-ebimas/lib/python3.10/site-packages/gymnasium/spaces/box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float32
  gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [21]:
from tqdm import tqdm

epochs = 30
num_steps = 1024
lengths = []

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = torch_deterministic

device = torch.device("cuda" if torch.cuda.is_available() and cuda else "cpu")

for epoch in tqdm(range(epochs)):
    next_obs, infos = reset_environment(env, device)

    next_obs = {k: torch.Tensor(v).unsqueeze(0).to(device) for k, v in next_obs.items()}
    next_done = {"sender": torch.zeros(num_envs).to(device), "receiver": torch.zeros(num_envs).to(device)}

    for step in range(0, num_steps):
        sender_action = sender.get_action_and_value(next_obs["sender"])[0]
        receiver_action = receiver.get_action_and_value(next_obs["receiver"])[0]

        next_obs, reward, terminations, truncations, info = env.step({"sender": sender_action.cpu().numpy()[0], "receiver": receiver_action.cpu().numpy()[0]})
        next_obs = {"sender": torch.Tensor(next_obs["sender"]).unsqueeze(0).to(device), "receiver": torch.Tensor(next_obs["receiver"]).unsqueeze(0).to(device)}

        if terminations["sender"] or terminations["receiver"] or truncations["sender"] or truncations["receiver"]:
            next_obs, infos = reset_environment(env, device)
            lengths.append(step)
            break
            
dynamic = env.env.env.environment_dynamics[3]
print("Accuracy:", sum(dynamic.accuracies) / len(dynamic.accuracies))
print("Variance:", sum(dynamic.variances) / len(dynamic.variances))
print("Send Accuracy:", sum(dynamic.sendAccuracies) / len(dynamic.sendAccuracies))
print("Length:", sum(lengths) / len(lengths))

100%|██████████| 30/30 [00:27<00:00,  1.10it/s]

Accuracy: 0.5
Variance: 1.0
Send Accuracy: 0.6164179104477612
Length: 63.96666666666667


In [22]:
dynamic = env.env.env.environment_dynamics[3]
print("Accuracy:", sum(dynamic.accuracies) / len(dynamic.accuracies))
print("Variance:", sum(dynamic.variances) / len(dynamic.variances))
print("Send Accuracy:", sum(dynamic.sendAccuracies) / len(dynamic.sendAccuracies))
print("Length:", sum(lengths) / len(lengths))

Accuracy: 0.5
Variance: 1.0
Send Accuracy: 0.6164179104477612
Length: 63.96666666666667
